In [1]:
import os
import os.path
import json
import numpy as np
import sys
import importlib
import tensorflow as tf
import string
import re

In [2]:
BASE_DIR = '/home/kalugin/Documents/experiments/pointnet-autoencoder'

In [3]:
ROOT_DIR = BASE_DIR
sys.path.append(BASE_DIR) # model
sys.path.append(ROOT_DIR) # provider
sys.path.append(os.path.join(ROOT_DIR, 'models'))
sys.path.append(os.path.join(ROOT_DIR, 'utils'))

In [4]:
MODEL = importlib.import_module('model')

In [5]:
DATA_PATH = os.path.join(BASE_DIR, 'data/shapenetcore_partanno_segmentation_benchmark_v0')

In [6]:
sys.path.append(os.path.join('./', 'utils'))

In [7]:
import show3d_balls
import tf_util

In [8]:
root = './data/shapenetcore_partanno_segmentation_benchmark_v0/'

In [9]:
with open(os.path.join(root, 'train_test_split', 'shuffled_test_file_list.json'), 'r') as f:
    test_ids = set([str(d.split('/')[2]) for d in json.load(f)])

In [10]:
catfile = os.path.join(root, 'synsetoffset2category.txt')

In [11]:
cat = {}

In [12]:
with open(catfile, 'r') as f:
    for line in f:
        ls = line.strip().split()
        cat[ls[0]] = ls[1]

In [13]:
item = 'Chair'

In [14]:
dir_point = os.path.join(root, cat[item], 'points')
dir_seg = os.path.join(root, cat[item], 'points_label')
fns = sorted(os.listdir(dir_point))
fns = [fn for fn in fns if fn[0:-4] in test_ids]

In [15]:
meta_chair = []

In [16]:
for fn in fns:
    token = (os.path.splitext(os.path.basename(fn))[0]) 
    meta_chair.append((os.path.join(dir_point, token + '.pts'), os.path.join(dir_seg, token + '.seg')))

In [17]:
points = [np.loadtxt(i[0]) for i in meta_chair[:100]]

In [23]:
show3d_balls.showpoints(points[0], ballradius=8)

KeyboardInterrupt: 

In [19]:
def placeholder_inputs(batch_size, num_point):
    pointclouds_pl = tf.placeholder(tf.float32, shape=(batch_size, num_point, 3))
    labels_pl = tf.placeholder(tf.float32, shape=(batch_size, num_point, 3))
    return pointclouds_pl, labels_pl

In [71]:
pointclouds_pl, is_training_pl = placeholder_inputs(1, 2048)
pred, end_points = MODEL.get_model(pointclouds_pl, is_training_pl)

TypeError: Input 'pred' of 'Switch' Op has type float32 that does not match expected type of bool.

In [ ]:
'''
            embedding = tf.placeholder(tf.float32, shape=(1, 1024))
            with tf.variable_scope('fc1', reuse=tf.AUTO_REUSE) as sc:
                net = tf_util.fully_connected(embedding, 1024, bn=True, is_training=False, scope='fc1', bn_decay=None)
            with tf.variable_scope('fc2', reuse=tf.AUTO_REUSE) as sc:
                net = tf_util.fully_connected(net, 1024, bn=True, is_training=False, scope='fc2', bn_decay=None)
            with tf.variable_scope('fc3', reuse=tf.AUTO_REUSE) as sc:
                net = tf_util.fully_connected(net, num_point*3, activation_fn=None, scope='fc3')
            net = tf.reshape(net, (batch_size, num_point, 3))
            '''

In [18]:
GPU_INDEX = 0

In [34]:
def get_model(MODEL_PATH, batch_size=1, num_point=2048):
    with tf.Graph().as_default():
        with tf.device('/gpu:'+str(GPU_INDEX)):
            pointclouds_pl, labels_pl = MODEL.placeholder_inputs(batch_size, num_point)
            is_training_pl = tf.placeholder(tf.bool, shape=())
            pred, end_points = MODEL.get_model(pointclouds_pl, is_training_pl)
            loss = MODEL.get_loss(pred, labels_pl, end_points)
            
            embedding = tf.placeholder(tf.float32, shape=(1, 1024))
            net = tf_util.fully_connected(embedding, 1024, bn=True, is_training=False, scope='fc1', bn_decay=None)
            net = tf_util.fully_connected(net, 1024, bn=True, is_training=False, scope='fc2', bn_decay=None)
            net = tf_util.fully_connected(net, num_point*3, activation_fn=None, scope='fc3')
            net = tf.reshape(net, (batch_size, num_point, 3))
            
            saver = tf.train.Saver()
        # Create a session
        print(net)
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        config.allow_soft_placement = True
        sess = tf.Session(config=config)
        # Restore variables from disk.
        saver.restore(sess, MODEL_PATH)
        ops = {'pointclouds_pl': pointclouds_pl,
               'labels_pl': labels_pl,
               'is_training_pl': is_training_pl,
               'embedding': embedding,
               'pred': pred,
               'loss': loss}
        return sess, ops, net

In [35]:
normal_sess, normal_ops, net_ = get_model('./log_chair_norotation/model_2.ckpt')

Tensor("Reshape_2:0", shape=(1, 2048, 3), dtype=float32, device=/device:GPU:0)
INFO:tensorflow:Restoring parameters from ./log_chair_norotation/model_2.ckpt


In [37]:
net_

<tf.Tensor 'Reshape_2:0' shape=(1, 2048, 3) dtype=float32>

In [36]:
normal_sess

In [46]:
pc_reconstruit = normal_sess.run(net_, feed_dict={normal_ops['embedding']: noised_emb})

In [49]:
pc_reconstruit[0].shape

(2048, 3)

In [50]:
show3d_balls.showpoints(points[1], ballradius=8)

KeyboardInterrupt: 

In [51]:
show3d_balls.showpoints(pc_reconstruit[0], ballradius=8)

KeyboardInterrupt: 

In [25]:
noised_sess, noised_ops = get_model('./log_chair_norotation/noised_model.ckpt')



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./log_chair_norotation/noised_model.ckpt


In [43]:
noised_emb.shape

(1, 1024)

In [65]:
noised_ops.keys()

['pred', 'is_training_pl', 'loss', 'pointclouds_pl', 'labels_pl']

In [36]:
ph_points = np.expand_dims(points[0][:2048, :], axis=0)

In [62]:
feed_dict = {noised_ops['pointclouds_pl']: np.expand_dims(points[1][:2048, :], axis=0), noised_ops['is_training_pl']: False}

In [40]:
noised_emb = normal_sess.run(normal_ops['loss'][1]['embedding'], feed_dict={normal_ops['pointclouds_pl']: np.expand_dims(points[1][:2048, :], axis=0),
                                                              normal_ops['is_training_pl']: False})

In [38]:
noised_emb = noised_sess.run(noised_ops['loss'][1]['embedding'], feed_dict={noised_ops['pointclouds_pl']: np.expand_dims(points[1][:2048, :], axis=0), 
                                                              noised_ops['labels_pl'] :np.expand_dims(points[1][:2048, :], axis=0),
                                                              noised_ops['is_training_pl']: False})

NameError: name 'noised_sess' is not defined

In [95]:
noised_emb

array([[1.1680152, 1.7588781, 1.3898653, ..., 2.431801 , 1.6550533,
        1.0289735]], dtype=float32)

In [85]:
emb = noised_emb[1]['embedding']

In [87]:
custom_emb = (0., {'embedding': emb, 'pcloss' : 0.}) 

In [90]:
MODEL.get_model()

<module 'model' from '/home/kalugin/Documents/experiments/pointnet-autoencoder/models/model.pyc'>

In [ ]:


output_labels = noised_sess.run(noised_ops['pred'], feed_dict={noised_ops['is_training_pl']: False, 
                                                               noised_ops['loss'] : custom_emb})

In [59]:
show3d_balls.showpoints(ph_points[0], ballradius=8)

KeyboardInterrupt: 

In [60]:
show3d_balls.showpoints(noised_emb[0][0], ballradius=8)

KeyboardInterrupt: 